# 🎥 Checkpoint 4 - Classical CV: Counting Yellow Obstacles

## 📌 Goal
You are given a video containing multiple objects of different colors.  
Your task is to **detect how many yellow obstacles are present in each frame** and output this as an array.

This assignment will help you practice **color-based segmentation, video processing, using Google Colab, and working through some corner cases** in computer vision.

---


## Step 0: Import Required Libraries

In [14]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Step 1: Mount your G Drive and access the videos

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, use the file explorer on the left to find the path to the videos for this task.

They should be in your UMARV Shared Drive called `STUDENT-Robotics | UMARV` at `2025-2026/CV Onboarding`

In [16]:
# TODO
train_video_path = "/content/drive/Shareddrives/STUDENT-Robotics | UMARV/2025-2026/CV Onboarding/Test-Checkpoint-CV.mp4"
test_video_path = "/content/drive/Shareddrives/STUDENT-Robotics | UMARV/2025-2026/CV Onboarding/Train-Checkpoint-CV.mp4"

## Step 2: Process the video

Read the frames of the train video using the OpenCV Fundamentals Onboarding section: [link](https://github.com/umigv/CV-Onboarding/blob/main/3_Programming/3_opencv.md)

In [3]:
cap = cv2.VideoCapture("video.mp4")

while cap.isOpened():
  ret, frame = cap.read();
  if not ret:
        break

  cv2.imshow("Frame", frame)

  if cv2.waitKey(1) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()
  pass

## Step 3: Get HSV range for Yellow

Use this stack overflow post to identify the HSV upper and lower limits for yellow: [link](https://stackoverflow.com/a/48367205)

The example snippet here might be helpful! [Link](https://github.com/umigv/CV-Onboarding/blob/main/4_Classical_CV/1_hsv_filtering.md)

The V values have already been put in for you. You need to fill the H and S values.

<details>
<summary>Hint 1</summary>
Are you seeing only small patches of yellow near the obstacles, but not the whole obstacle in your mask? Try decreasing the lower limit of H further to include brownish hues as well.
</details>

<details>
<summary>Hint 2</summary>
Are you seeing the yellow lane lines in your mask? Try increasing the upper limit of S further to only look at darker yellows.
</details>

<details>
<summary>Final Hint!</summary>
<ul>
<li>Range for H: (5,40) </li>
<li>Range for S: (100,255) </li>
</ul>
</details>

In [24]:
lower_yellow = np.array([5, 40, 20])   # lower bound [H, S, V]
upper_yellow = np.array([40, 255, 255])  # upper bound [H, S, V]

## Step 4: Core Logic for Yellow Obstacle Detection

Use HSV filtering [(link)](https://github.com/umigv/CV-Onboarding/blob/main/4_Classical_CV/1_hsv_filtering.md) and Contours [(link)](https://github.com/umigv/CV-Onboarding/blob/main/4_Classical_CV/2_contours.md) to identify the yellow obstacles and count how many there are in each frame!

This list of features will be useful in this task (especially **Contour Area**): [Contour Features](https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html)

<details>
<summary>Hint 1</summary>
You should be able to copy paste the code for `hsv = ...` and `mask = ...` from the onboarding docs for HSV filtering.
</details>

<details>
<summary>Hint 2</summary>
You should be able to copy paste the code for contours for `contours = ...` from the onboarding docs for contours.
</details>

<details>
<summary>Hint 3</summary>
You should be able to iterate over `contours`, which is a list of contours. Use `cv2.contourArea(cnt)` to know its area. Thresholding area to find significant contours will help! That is, `if cv2.contourArea(cnt) > THRESHOLD`, do something.
</details>

<details>
<summary>Final Hint!</summary>
`if cv2.contourArea(c) > 1100:`, increment your counter for yellow obstacles.
</details>

In [21]:
from google.colab.patches import cv_imshow

# TODO
def count_yellow_obstacles(frame):
  # 1. Apply HSV Filtering on the frame to obtain a mask of yellow objects
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
  mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

  # 2. Use Contours to identify these objects in the mask (remember, all other objects will be masked out!)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  # 3. Use some logic (like the area or shape of the contour to detect obstacles)
  yellow_obstacle_count = 0
  for cnt in contours:
    if cv2.contourArea(cnt) > 1100:
      yellow_obstacle_count += 1
  # Visualization
  frame_resized = cv2.resize(frame, (300, 300))
  mask_resized  = cv2.resize(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), (300, 300))
  side_by_side  = np.hstack((frame_resized, mask_resized))
  cv_imshow(side_by_side)


  # 4. Return a count of yellow obstacles identified
  return yellow_obstacle_count

## Step 5: Integrate everything into `main()`

In any program, the `main()` or `run()` function serves as the entry point to the rest of the functionality. So that's what we'll implement as well.

In [25]:
def main(video_path=train_video_path):
  cap = cv2.VideoCapture(video_path)
  yellow_obstacle_count_list = []
  target_fps = 5
  input_fps = cap.get(cv2.CAP_PROP_FPS)
  frame_interval = int(round(input_fps / target_fps))

  frame_count = 0
  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      if frame_count % frame_interval == 0:
        yellow_obstacle_count = count_yellow_obstacles(frame)
        yellow_obstacle_count_list.append(yellow_obstacle_count)

      frame_count += 1

  print(len(yellow_obstacle_count_list))
  print(yellow_obstacle_count_list)
  plt.plot(yellow_obstacle_count_list)


main()



Output hidden; open in https://colab.research.google.com to view.

## Step 6: Test your algorithm on the Test Video

In [26]:
main(video_path=test_video_path)

Output hidden; open in https://colab.research.google.com to view.